In [1]:
import sys
sys.path.append('/fsx/matzeni/duck')

In [2]:
import torch
import h5py
import json
from pathlib import Path
import pickle
from tqdm import tqdm
import logging
from transformers import AutoTokenizer
from typing import Any, Dict, List, Optional, Tuple
import copy
from einops import rearrange, repeat
import numpy as np
from duck.box_tensors import BoxTensor
from duck.task.duck_entity_disambiguation import Duck
from hydra import compose, initialize
import hydra
from duck.common.utils import make_reproducible
import collections
import logging
import math
logger = logging.getLogger()

In [3]:
initialize(config_path="conf", version_base=None)

hydra.initialize()

In [4]:
ckpt_path = "/checkpoints/matzeni/duck/checkpoints/duck_conditional_softmax_epoch=8_duck_loss=1.946.ckpt"
duck = Duck.load_from_checkpoint(ckpt_path).eval().cuda()

ConfigAttributeError: Missing key loss
    full_key: loss
    object_type=dict

In [ ]:
config = compose(
    config_name="duck_conf",
    overrides=[]
)

In [ ]:
datamodule = hydra.utils.instantiate(config.data)

In [ ]:
def batch_to_cuda(batch):
    if isinstance(batch, torch.Tensor):
        return batch.cuda()
    if isinstance(batch, collections.abc.Mapping):
        return {k: batch_to_cuda(v) for k, v in batch.items()}
    if isinstance(batch, str):
        return batch
    if isinstance(batch, collections.abc.Sequence):
        return [batch_to_cuda(v) for v in batch]
    return batch

In [ ]:
make_reproducible(ngpus=config.trainer.devices)
train_data = datamodule.train_dataloader()
train_data_iterator = iter(train_data)

In [ ]:
batch = next(train_data_iterator)
batch = next(train_data_iterator)
batch = batch_to_cuda(batch)
representations = duck(batch)
mentions = representations["mentions"]
entities = representations["entities"]
entity_boxes = representations["entity_boxes"]
neighbors = representations["neighbors"]
neighbor_boxes = representations["neighbor_boxes"]
entity_relation_ids = batch["relation_ids"]
neighbor_relation_ids = batch["neighbor_relation_ids"]
neighbors, neighbor_boxes, neighbor_relation_ids = duck._extend_with_in_batch_neighbors(
    neighbors,
    neighbor_boxes,
    neighbor_relation_ids
)        

In [ ]:
print(batch["entity_labels"])

In [ ]:
entity_boxes

In [ ]:
from duck.common.utils import load_json

In [ ]:
ent_to_rel = load_json("/fsx/matzeni/data/duck/ent_to_rel.json")

In [ ]:
all_rels = set(r for rels in ent_to_rel.values() for r in rels)

In [ ]:
rel_to_ent = {r: set() for r in rels}
for e, rels in tqdm(ent_to_rel.items()):
    for r in rels:
        rel_to_ent[r].add(e)

In [ ]:
"Cristiano Ronaldo" in rel_to_ent["P36"]

In [ ]:
all_rels = list(all_rels)
pmi = {}
eps = 1e-14
for i, r1 in enumerate(tqdm(all_rels)):
    for j in range(i, len(all_rels)):
        r2 = all_rels[j]
        ents1 = rel_to_ent[r1]
        ents2 = rel_to_ent[r2]
        p_r1 = len(ents1) / len(ent_to_rel)
        p_r2 = len(ents2) / len(ent_to_rel)
        p_r1r2 = len(ents1 & ents2) / len(ent_to_rel)
        pmi[(r1, r2)] = math.log(p_r1r2 + eps) - math.log(p_r1 + eps) - math.log(p_r2 + eps)
        

In [ ]:
len(ent_to_rel)

In [ ]:
mse = torch.rand((32, 7, 2))

In [ ]:
a, b = zip(*[(0,1), (1, 2), (3, 3)])
list(a)

In [ ]:
mse[list(a), list(b)] = 0.

In [ ]:
mse